In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.fftpack import dct, idct
from scipy.optimize import minimize
sns.set_theme()

## Solve: 
$$
\hat{s} = \text{argmin} ||s|| 
\\
\text{s.t.}
\\
\begin{align*}
y &= \Theta s \\
&= C \Psi s
\end{align*}

$$

In [ ]:
def cosamp(sampling_matrix, sample_vector, s, epsilon=1e-10, max_iter=1000):
    """
    Return an `s`-sparse approximation of the target signal
    Input:
        - sampling_matrix, sampling matrix Phi [n_samples x n_signal]
        - sample_vector, noisy sample vector
        - s, sparsity
    """
    n_samples, n_signal = sampling_matrix.shape
    best_signal_approx = np.zeros(sampling_matrix.shape[1])
    reconstr_error = sample_vector.copy()
    it = 0 # count
    halt = False
    while not halt:
        it += 1
        print("Iteration {}\r".format(it), end="")
        
        transformed_signal = np.dot(np.transpose(sampling_matrix), reconstr_error) # reconstruct signal in cosine transformed coordinates 
        omega = np.argsort(transformed_signal)[-(2*s):] # get index of largest/most important components
        omega = np.union1d(omega, best_signal_approx.nonzero()[0]) # use set instead?
        sampling_matrixT = sampling_matrix[:, omega]
        signal_approx = np.zeros(n_signal)  # initiate sparse approx
        # Solve Least Square
        signal_approx[omega], _, _, _ = np.linalg.lstsq(sampling_matrixT, sample_vector) #(solve x: sample_vector = sampling_matrixT @ x)
        
        # Get new estimate
        signal_approx[np.argsort(signal_approx)[:-s]] = 0 # remove smallest entries
        best_signal_approx = signal_approx.copy()
        
        # Halt criterion
        reconstr_error_prev = reconstr_error.copy()
        reconstr_error = sample_vector - np.dot(sampling_matrix, best_signal_approx)

        halt = (np.linalg.norm(reconstr_error - reconstr_error_prev) < epsilon) or \
            np.linalg.norm(reconstr_error) < epsilon or \
            it > max_iter
        
    return best_signal_approx

In [ ]:
np.random.seed(0)
n = 4096
t = np.linspace(0, 1, n)
signal = np.cos(2 * 97 * np.pi * t) + np.cos(2 * 777 * np.pi * t)
noise = np.random.normal(0, 0.5, len(t))
x = signal + noise
xt = np.fft.fft(x)
PSD = xt * np.conj(xt) / n #Power spectral denstiy, ||xt||^2 / n

In [ ]:
# Randomly sampled signal
num_samples = 128 
permutation = np.floor(np.random.rand(num_samples) * n).astype(int)
compressed_signal = x[permutation]

In [ ]:
# Solve compressed sensing problem
Psi = dct(np.identity(n))
Theta = Psi[permutation, :]

s = cosamp(Theta, compressed_signal, 10, epsilon = 1.e-10, max_iter = 10)
x_reconstructed = idct(s)
xt_reconstructed = np.fft.fft(x_reconstructed, n)
PSD_reconstructed = xt_reconstructed * np.conj(xt_reconstructed) / n

In [ ]:
time_window = np.array([1024, 1280])/n
freq = np.arange(n)
L = int(np.floor(n/2))

fig, ax = plt.subplots(2,2, figsize = (12, 8))
ax = ax.reshape(-1)

ax[0].plot(t, x, '.-', color = 'black', label = 'signal + noise')
ax[0].plot(t, signal, 'm', linewidth = 2, label = 'true signal')
ax[0].plot(permutation/n, compressed_signal, marker = 'o', color = 'r', ms = 12, linewidth = 0, label = 'compressed signal')
ax[0].set_xlim(time_window[0], time_window[1]) 
ax[0].set_xlabel('Time [s]')
ax[0].legend()

ax[1].plot(freq[:L], PSD[:L].real, color = 'k')
ax[1].set_xlim(0, 1024)
ax[1].set_xlabel('Frequency [Hz]')

ax[2].plot(t, signal, 'm', label = 'true signal')
ax[2].plot(t, x_reconstructed, '.-', color = 'c')
ax[2].set_xlim(time_window[0], time_window[1])
ax[2].set_ylim(ax[0].get_ylim())
ax[2].set_xlabel(ax[0].get_xlabel())
ax[2].set_ylabel('Reconstructed signal')

ax[3].plot(freq[:L], PSD_reconstructed[:L], color = 'c')
ax[3].set_xlim(ax[1].get_xlim())
ax[3].set_ylim(ax[1].get_ylim())
ax[3].set_xlabel(ax[1].get_xlabel())
ax[3].set_ylabel(ax[1].get_ylabel())